In [157]:
text_file = open("trainset.txt", "r")
lines = text_file.readlines()
#print(lines[1])
text_file.close()
#read line by line

In [277]:
listOfAttributes = [] # implicit instantiation
listOfAttributes.append([])
i = 0
while i < len(lines):
    attributes = lines[i].split('\t') #split it by tab
    listOfAttributes.append([])
    listOfAttributes[i].append(attributes[0])
    listOfAttributes[i].append(attributes[1])
    listOfAttributes[i].append(attributes[2])
    listOfAttributes[i].append(attributes[3])
    i+=1
        
#print(listOfAttributes[0])
# print(len(listOfAttributes) )

### Prepossesing

In [294]:
#Doing prepossesing
import re #regex
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

def preprocess(eliment):
      
    eliment = re.sub(r'([^\s\w]|_)+', '', eliment) # Remove all the special characters except space
    eliment = eliment.lower()
    eliment = re.sub(r'\s+', ' ', eliment, flags=re.I)# Substituting multiple spaces with single space

    all_words = eliment.split()
    en_stops = set(stopwords.words('english')) #stopword handle
    eliment = ""
    ps = PorterStemmer() #stemming handle  

    for word in all_words: 
        if word not in en_stops:   
            eliment +=  ps.stem(word) +" "

    return eliment 

listOfAttributesProcessed = [] # implicit instantiation
listOfAttributesProcessed.append([])
i = 0
while i <len(lines):
    listOfAttributesProcessed.append([])
    listOfAttributesProcessed[i].append(listOfAttributes[i][0])
    
    processedEliment1= preprocess(listOfAttributes[i][1] )
    processedEliment2= preprocess(listOfAttributes[i][2] )
    processedEliment3= preprocess(listOfAttributes[i][3] )
    
    AllEliments = processedEliment1+processedEliment2+processedEliment3 #combine attributes
    
    listOfAttributesProcessed[i].append(AllEliments)

#    print(listOfAttributesProcessed[i])  
    i+=1

In [187]:
import csv

with open("featuresTrain.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(listOfAttributesProcessed)

In [290]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

df=pd.read_csv('featuresTrain.csv',sep=',',names=['class','text'])
df.head();

### Traning and testing data extraction

In [296]:
df_x=df["text"]
df_y=df["class"]
cv = TfidfVectorizer(min_df=1)
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.333, random_state=0)


In [297]:
cv = TfidfVectorizer(max_features=1500, min_df=1, max_df=0.7) #minimum number of documents that should contain this feature
x_traincv=cv.fit_transform(x_train)
a=x_traincv.toarray()

### Predictions

In [342]:
mnb = MultinomialNB()
mnb.fit(x_traincv,y_train);
#x_testcv=cv.transform(x_test);
x_testcv.toarray();
predictions=mnb.predict(x_testcv)
#print(predictions);

### Accuracies

In [298]:
actual=np.array(y_test)
print('Training Accuracy: ',mnb.score(x_traincv,y_train))

Training Accuracy:  1.0


In [281]:
# calculate accuracy of class predictions
from sklearn import metrics
acc = metrics.accuracy_score(actual, predictions)
print('Testing Accuracy: ',acc)

Testing Accuracy:  0.9552238805970149


### Confution matrix

In [343]:
print(metrics.confusion_matrix(actual, predictions)) #confusion matrix

[[23  0]
 [ 3 41]]


Confusion matrix 
[TN FP
 FN TP]

### Classification report

In [344]:
target_names =["+1", "-1"]
print(classification_report(actual, predictions, target_names=target_names))

             precision    recall  f1-score   support

         +1       0.88      1.00      0.94        23
         -1       1.00      0.93      0.96        44

avg / total       0.96      0.96      0.96        67



### testsetwithoutlabels.txt prediction

In [345]:
text_file = open("testsetwithoutlabels.txt", "r")
lines = text_file.readlines()
#print(lines)
test=cv.transform(lines);
test.toarray();
predictions=mnb.predict(test)
print(predictions)

[ 1  1  1 -1 -1  1 -1 -1 -1 -1 -1 -1  1  1 -1  1  1  1  1  1 -1 -1 -1 -1
 -1 -1  1 -1 -1 -1 -1  1 -1  1 -1  1 -1  1 -1 -1 -1  1 -1 -1  1 -1  1 -1
  1 -1  1  1 -1 -1  1 -1  1  1  1 -1 -1 -1 -1 -1  1 -1  1  1  1 -1  1 -1
 -1 -1  1 -1 -1  1  1 -1  1 -1  1 -1  1 -1 -1  1  1  1  1  1  1 -1  1 -1
 -1  1 -1  1]
